<a href="https://colab.research.google.com/github/forexms78/AI-05-/blob/main/transformer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import math
import numpy as np

In [4]:
#def scaled_dot_product_attention

def scaled_dot_product_attention(q, k, v, mask=None):

  matmul_qk = torch.matmul(q, k.transpose(-1, -2)) #곱하기 쿼리하고 키

  dk = q.size(-1)
  scaled_attention_logits = matmul_qk / math.sqrt(dk) #스케일링

  if mask is not None:
    scaled_attention_logits = scaled_attention_logits.masked_fill(mask == False, float('-1e9'))

  #softmax
  attention_weights = F.softmax(scaled_attention_logits, dim=-1)

  output = torch.matmul(attention_weights, v)

  return output, attention_weights

#test code
x = torch.randn(3,20,64)
out, atw = scaled_dot_product_attention(x,x,x, mask=None)

print(f'attention output shape : {out.shape}')
print(f'attention weight shape : {atw.shape}')

attention output shape : torch.Size([3, 20, 64])
attention weight shape : torch.Size([3, 20, 20])


In [8]:
# @title multihead attention

class MultiHeadAttention(nn.Module):
  def __init__(self, em_dim, num_heads):
    super(MultiHeadAttention, self).__init__()

    self.em_dim = em_dim # 임베딩 차원 설정
    self.num_heads = num_heads # 어텐션 헤드 개수 설정

    self.head_dim = em_dim // num_heads # 각 어텐션 헤드의 차원 계산

    self.wq = nn.Linear(em_dim, em_dim) # 쿼리 변환을 위한 선형 레이어 정의
    self.wk = nn.Linear(em_dim, em_dim) # 키 변환을 위한 선형 레이어 정의
    self.wv = nn.Linear(em_dim, em_dim) # 값 변환을 위한 선형 레이어 정의

    self.dense = nn.Linear(em_dim, em_dim) # 최종 출력을 위한 선형 레이어 정의


  #split_heads
  def split_heads(self, x):
    batch_size, seq_len, em_dim = x.size() # 입력 텐서의 배치 크기, 시퀀스 길이, 임베딩 차원 가져오기
    x = x.view(batch_size, seq_len, self.num_heads, self.head_dim) # 어텐션 헤드 개수와 헤드 차원에 맞춰 텐서 모양 변경
    return x.permute(0, 2, 1, 3) # 어텐션 계산을 위해 차원 순서 변경 (batch_size, num_heads, seq_len, head_dim)

  #forward
  def forward(self, q, k, v, mask=None):
    batch_size = q.size(0) # 배치 크기 가져오기

    q = self.wq(q) # 쿼리 선형 변환 적용
    k = self.wk(k) # 키 선형 변환 적용
    v = self.wv(v) # 값 선형 변환 적용

    q = self.split_heads(q) # 쿼리를 여러 헤드로 분할
    k = self.split_heads(k) # 키를 여러 헤드로 분할
    v = self.split_heads(v) # 값을 여러 헤드로 분할

    scaled_attention, attention_weights = scaled_dot_product_attention(q, k, v, mask) # 스케일드 닷 프로덕트 어텐션 계산

    scaled_attention = scaled_attention.permute(0,2,1,3).contiguous() # 어텐션 결과를 원래 차원 순서로 되돌리기
    concat_attention = scaled_attention.view(batch_size, -1, self.em_dim) # 여러 헤드의 어텐션 결과를 하나로 연결

    output = self.dense(concat_attention) # 최종 선형 변환 적용

    return output, attention_weights # 어텐션 결과와 어텐션 가중치 반환

# test
x = torch.randn(2,10,64)
mh = MultiHeadAttention(em_dim=64, num_heads=2)
out, atw = mh(x,x,x)
print(f'attention output shape : {out.shape}')
print(f'attention weight shape : {atw.shape}')

attention output shape : torch.Size([2, 10, 64])
attention weight shape : torch.Size([2, 2, 10, 10])
